# Statistics of data

In [2]:
import  pandas as pd
df15 = pd.read_csv("trainLabels15.csv")
df_pivot = df15.pivot(columns='level', values='image')
df_pivot.count()

level
0    25810
1     2443
2     5292
3      873
4      708
dtype: int64

In [3]:
df19 = pd.read_csv("trainLabels19.csv")
df_pivot = df19.pivot(columns='diagnosis', values='id_code')
df_pivot.count()

diagnosis
0    1805
1     370
2     999
3     193
4     295
dtype: int64

The data exhibits a class imbalance, where class 0 has a significantly higher number of images compared to classes 1, 2, 3, and 4. This can negatively impact machine learning models by causing them to favor the majority class (class 0) during training.

### Addressing Class Imbalance
The proposed strategy to counter this imbalance is to rotate the images in classes 1, 2, 3, and 4. This would create new images that could be added to respective classes. Specifically, the strategy suggests rotating the images by 90 degrees, 180 degrees, and 270 degrees. This would triple the number of images in each of the classes being rotated.


## Merging 2019 and 2015 datasets

In [4]:
df19.rename(columns = {'id_code':'image', 'diagnosis':'level'}, inplace = True)
df19

,image,level
0,000c1434d8d7,2
1,001639a390f0,4
2,0024cdab0c1e,1
3,002c21358ce6,0
4,005b95c28852,0
...,...,...
3657,ffa47f6a7bf4,2
3658,ffc04fed30e6,0
3659,ffcf7b45f213,2
3660,ffd97f8cd5aa,0


In [5]:
df = pd.merge(df15, df19, how='outer')
df_pivot = df.pivot(columns='level', values='image')
df_pivot.count()

level
0    27615
1     2813
2     6291
3     1066
4     1003
dtype: int64

# Datasets
The two datasets are now merged in one dataframe
further actions
- **Generate CSV**: Create a CSV file containing the merged dataframe for easy access and reference.
- **Organizing the images**: Organize the images into their respective directories based on their class labels for efficient management and access.
- **Augmentation of Images**: Rotate images belonging to classes 1, 2, 3, and 4 to counter the imbalance within these classes, ensuring a more balanced representation across all classes.

In [6]:
df.to_csv("train19&15.csv", index = False)

In [11]:
train_Images = df['image'].count()

##### Dividing images to respective directories according to their classes

In [17]:
import os
import shutil
from tqdm.notebook import tqdm


for label in df['level'].unique():
    class_dir = os.path.join('class ' + str(label))
    if not os.path.exists(class_dir):
        os.makedirs(class_dir)

for index, row in tqdm(df.iterrows(), total=len(df)):
    image_name = row['image'] + '.jpg'
    class_label = row['level']

    for directory in ['../../../Downloads/archive/resized train 15', '../../../Downloads/archive/resized train 19']:
        if image_name in os.listdir(directory):
            image_to_copy = os.path.join(directory, image_name)
            destination = os.path.join('class ' + str(class_label))
            shutil.copy(image_to_copy, destination)
            break
        if directory == '../../../Downloads/archive/resized train 19':
            print("IMAGE NOT FOUND")

  0%|          | 0/38788 [00:00<?, ?it/s]

##### Augmenting the images of class 1, 2, 3, 4

In [18]:
from tqdm.notebook import tqdm
import cv2

Augmenting_df = df[df['level'] != 0]
Augmented_df = df

for index, row in tqdm(Augmenting_df.iterrows(), total = len(Augmenting_df)):
    image_name = row['image'] + '.jpg'
    class_label = row['level']
    directory = 'class ' + str(class_label)
    Original_Image = cv2.imread(str(os.path.join(directory, image_name)))
    rotated_image1 = cv2.rotate(Original_Image, cv2.ROTATE_180)
    rotated_image2 = cv2.rotate(Original_Image, cv2.ROTATE_90_CLOCKWISE)
    rotated_image3 = cv2.rotate(Original_Image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    for image, name in [(rotated_image1, row['image'] + '_rotate180'), (rotated_image2, row['image'] + '_rotate270' ),
                        (rotated_image3, row['image'] + '_rotate90')]:
         
        new_index= Augmented_df.index[-1] + 1
        rotated_row = pd.DataFrame({'image' : name, 'level' : class_label}, index = [new_index] )
        Augmented_df = pd.merge(Augmented_df, rotated_row, how='outer')
        cv2.imwrite(str(os.path.join(directory, name)), image)

Augmented_df.to_csv("AugmentedTrain19&15.csv", index = False)

  0%|          | 0/11173 [00:00<?, ?it/s]

In [19]:
Augmented_df

,image,level
0,000c1434d8d7,2
1,000c1434d8d7_rotate180.jpg,2
2,000c1434d8d7_rotate270.jpg,2
3,000c1434d8d7_rotate90.jpg,2
4,001639a390f0,4
...,...,...
72302,ffd97f8cd5aa,0
72303,ffec9a18a3ce,2
72304,ffec9a18a3ce_rotate180.jpg,2
72305,ffec9a18a3ce_rotate270.jpg,2


In [20]:
df_pivot = Augmented_df.pivot(columns='level', values='image')
df_pivot.count()

level
0    27615
1    11252
2    25164
3     4264
4     4012
dtype: int64

In [34]:
Augmented_df.to_csv("AugmentedTrain19&15.csv", index = False)

# Applying SMOTE

As seen above the imbalance in classes is still very high even after we augmented them, since GANs produce some faulty images in terms of fundus photography, we have to move towards SMOTE to balance the classes to finally move towards model training.

In [150]:
from imblearn.over_sampling import SMOTE

X = Augmented_df
y =Augmented_df['level']

name_to_index = dict(enumerate(X.image))
index_to_name = {v: k for k, v in name_to_index.items()}

X.drop('level', axis = 1)
def replace_values(x):
    return index_to_name.get(x)

X = X['image'].apply(replace_values)
print(X.dtype)
X = pd.DataFrame({'image': X})

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

resampled_data = pd.DataFrame(X_resampled, columns=['image'])
resampled_data['level'] = y_resampled

resampled_data

int64


,image,level
0,0,2
1,1,2
2,2,2
3,3,2
4,4,4
...,...,...
138070,69196,4
138071,31266,4
138072,71881,4
138073,47465,4


In [172]:
def replace_indices_with_original_names(x):
    return name_to_index.get(x)

resampled_data['image'] = resampled_data['image'].apply(replace_indices_with_original_names)

resampled_data


,image,level
0,000c1434d8d7,2
1,000c1434d8d7_rotate180.jpg,2
2,000c1434d8d7_rotate270.jpg,2
3,000c1434d8d7_rotate90.jpg,2
4,001639a390f0,4
...,...,...
138070,a76b69e443ce_rotate180.jpg,4
138071,28948_left_rotate90.jpg,4
138072,f2094a20b275_rotate270.jpg,4
138073,38856_left_rotate270.jpg,4


In [176]:
resampled_data.to_csv('BalancedDataSMOTEwithRelativeNames.csv', index = False)